In [22]:
import pandas as pd

# Load the JSON file into a Pandas DataFrame
salesdf = pd.read_json('mongodb_sample_supplies.json', lines=True)

In [23]:
salesdf.head()

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"[{'name': 'laptop', 'tags': ['electronics', 's...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone
1,{'$oid': '5bd761dcae323e45a93cd000'},{'$date': '2015-05-15T13:43:24.561Z'},"[{'name': 'printer paper', 'tags': ['office', ...",London,"{'gender': 'F', 'age': 27, 'email': 'zu@bainku...",False,Online
2,{'$oid': '5bd761dcae323e45a93ccfeb'},{'$date': '2015-02-23T09:53:59.343Z'},"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store
3,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online
4,{'$oid': '5bd761dcae323e45a93ccfe9'},{'$date': '2015-08-25T10:01:02.918Z'},"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone


In [24]:
#Flatten the 'items' column into individual rows
salesdf = salesdf.explode('items')

# Get the product name from 'items' dictionary
salesdf['product_name'] = salesdf['items'].apply(lambda x: x['name'])

# Group by product name and count the occurrences
top_products = salesdf['product_name'].value_counts().nlargest(10).reset_index()

# Rename the columns for clarity
top_products.columns = ['Product Name', 'Count']

print(top_products)


    Product Name  Count
0        notepad   6872
1         binder   4676
2           pens   4559
3      envelopes   4505
4       backpack   2309
5         laptop   2271
6  printer paper   2246


In [28]:
# Calculate the sales (quantity x price) for each product
salesdf['sales'] = salesdf['items'].apply(lambda x: x['quantity'] * float(x['price']['$numberDecimal']))

# Group by store location and product name, and sum the sales
top_3_products_by_store = salesdf.groupby(['storeLocation', salesdf['items'].apply(lambda x: x['name'])])['sales'].sum().reset_index()

# Rank the products by sales for each store location
top_3_products_by_store['rank'] = top_3_products_by_store.groupby('storeLocation')['sales'].rank(ascending=False, method='min')

# Filter the top 3 products for each store location
top_3_products_by_store = top_3_products_by_store[top_3_products_by_store['rank'] <= 3]

print(top_3_products_by_store)


   storeLocation     items       sales  rank
0         Austin  backpack   115217.59   2.0
3         Austin    laptop  1018494.05   1.0
5         Austin      pens    74622.52   3.0
7         Denver  backpack   245679.34   2.0
10        Denver    laptop  1961659.25   1.0
12        Denver      pens   185008.39   3.0
14        London  backpack   144529.14   2.0
17        London    laptop  1073098.72   1.0
19        London      pens    92602.30   3.0
21      New York  backpack    82563.14   2.0
24      New York    laptop   694613.32   1.0
26      New York      pens    64029.90   3.0
28     San Diego  backpack    50997.42   2.0
31     San Diego    laptop   462973.58   1.0
33     San Diego      pens    42134.55   3.0
35       Seattle  backpack   178387.47   2.0
38       Seattle    laptop  1565138.15   1.0
40       Seattle      pens   123445.61   3.0


In [32]:
import pandas as pd

# Load the JSON file into a Pandas DataFrame
salesdf = pd.read_json('mongodb_sample_supplies.json', lines=True)

# Flatten the 'items' column into individual rows
salesdf = salesdf.explode('items')

# Define a custom lambda function to calculate sales
def calculate_sales(item):
    price = float(item['price']['$numberDecimal'])
    return item['quantity'] * price

# Apply the custom lambda function to calculate sales
salesdf['sales'] = salesdf['items'].apply(calculate_sales)

# Group by store location and sum the sales for each location
store_rankings = salesdf.groupby('storeLocation')['sales'].sum().reset_index()

# Rank the store locations by sales
store_rankings['rank'] = store_rankings['sales'].rank(ascending=False, method='min')

# Sort the store rankings by rank
store_rankings = store_rankings.sort_values(by='rank')

print(store_rankings)


  storeLocation       sales  rank
1        Denver  2921009.92   1.0
5       Seattle  2255947.69   2.0
2        London  1583066.79   3.0
0        Austin  1445603.11   4.0
3      New York  1016059.59   5.0
4     San Diego   672885.17   6.0


In [42]:
print(salesdf.columns)

Index(['_id', 'saleDate', 'items', 'storeLocation', 'customer', 'couponUsed',
       'purchaseMethod', 'sales'],
      dtype='object')


In [64]:

# Extract gender information from the 'customer' column
salesdf['gender'] = salesdf['customer'].apply(lambda x: x.get('gender'))

# Create a pivot table to count the occurrences of each purchase method by gender
purchase_method_by_gender = pd.crosstab(salesdf['gender'], salesdf['purchaseMethod'])

# Reset the index to have gender as a column, not an index
purchase_method_by_gender = purchase_method_by_gender.reset_index()

# Drop the 'purchaseMethod' columns
purchase_method_by_gender = purchase_method_by_gender.drop(['Online', 'In-Store', 'Phone'], axis=1)

print(purchase_method_by_gender)

KeyError: "['purchaseMethod'] not found in axis"